In [1]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create a table to store data from multiple sources
cursor.execute('''
CREATE TABLE IF NOT EXISTS calendar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER NOT NULL,
    month INTEGER NOT NULL,
    day INTEGER NOT NULL,
    week_number INTEGER NOT NULL,
    quarter INTEGER NOT NULL,
    day_of_week TEXT NOT NULL,
    is_weekend BOOLEAN NOT NULL,
    full_date DATETIME NOT NULL,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')



# Commit changes and close the connection
conn.commit()
conn.close()

print("Database and table created successfully.")

Database and table created successfully.


In [2]:
from datetime import datetime, timedelta

# Generate records from 1900 to 2030
start_date = datetime(1900, 1, 1)
end_date = datetime(2030, 12, 31)
current_date = start_date

records = []
while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    week_number = current_date.isocalendar()[1]
    quarter = (month - 1) // 3 + 1
    day_of_week = current_date.strftime('%A')
    is_weekend = day_of_week in ['Saturday', 'Sunday']
    full_date = current_date.strftime('%Y-%m-%d')
    
    records.append((year, month, day, week_number, quarter, day_of_week, is_weekend, full_date))
    current_date += timedelta(days=1)

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Insert records into the calendar table
cursor.executemany('''
INSERT INTO calendar (year, month, day, week_number, quarter, day_of_week, is_weekend, full_date)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', records)

# Commit changes
conn.commit()

# Close the connection after the operation
conn.close()

print(f"{len(records)} records inserted into the calendar table.")

47847 records inserted into the calendar table.


In [3]:
import fastf1

# Enable the cache for fastf1
import os

# Ensure the cache directory exists
cache_dir = 'fastf1_cache'
os.makedirs(cache_dir, exist_ok=True)

# Enable the cache for fastf1
fastf1.Cache.enable_cache(cache_dir)

# Initialize an empty list to store F1 calendar data
f1_calendar_data = []

# Loop through the years from 2000 to 2025
for year in range(2000, 2026):
    # Load the F1 schedule for the current year
    schedule = fastf1.get_event_schedule(year)
    
    # Extract relevant data for the table
    for _, event in schedule.iterrows():
        race_name = event['EventName']
        country = event['Country']
        circuit = event['Location']
        event_year = event['EventDate'].year
        date = event['EventDate'].strftime('%Y-%m-%d')
        f1_calendar_data.append((race_name, country, circuit, event_year, date))

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the f1_calendar table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS fone_calendar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    race_name TEXT NOT NULL,
    country TEXT NOT NULL,
    circuit TEXT NOT NULL,
    year INTEGER NOT NULL,
    date TEXT NOT NULL
)
''')

# Insert F1 calendar data into the table
cursor.executemany('''
INSERT INTO fone_calendar (race_name, country, circuit, year, date)
VALUES (?, ?, ?, ?, ?)
''', f1_calendar_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print("F1 calendar data from 2000 to 2025 inserted into the database successfully.")


F1 calendar data from 2000 to 2025 inserted into the database successfully.


In [ ]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the fone_geography table
cursor.execute('''
CREATE TABLE IF NOT EXISTS fone_geography (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    circuit TEXT NOT NULL,
    city TEXT NOT NULL,
    country TEXT NOT NULL,
    continent TEXT NOT NULL,
    latitude REAL NOT NULL,
    longitude REAL NOT NULL,
    existing INTEGER DEFAULT 0
)
''')

# Define a mapping of circuits to their geographic information
# Define map for same circuits
same_circuits = {
    'Monte Carlo': ['Monaco', 'Monte Carlo','Monte-Carlo'],
    'Marina Bay': ['Singapore', 'Marina Bay'],
    'Catalunya': ['Montmeló', 'Barcelona','Spain'],
    'Spa-Francorchamps': ['Spa', 'Stavelot','Spa-Francorchamps'],
    'Montreal': ['Montréal', 'Montreal'],
    'Nürburgring': ['Nürburg', 'Nürburgring'],
    'Yas Marina': ['Yas Island', 'Abu Dhabi', 'Yas Marina'],
    'Sakhir': ['Bahrain', 'Sakhir']
    }
geography_data = [
    ('Silverstone', 'Silverstone', 'United Kingdom', 'Europe', 52.0786, -1.0169, 1),
    ('Budapest', 'Mogyoród', 'Hungary', 'Europe', 47.5789, 19.2486, 1),
    ('Monza', 'Monza', 'Italy', 'Europe', 45.6156, 9.2811, 1),
    ('São Paulo', 'São Paulo', 'Brazil', 'South America', -23.7036, -46.6997, 1),
    ('Sakhir', 'Sakhir', 'Bahrain', 'Asia', 26.0325, 50.5106, 1),
    ('Melbourne', 'Melbourne', 'Australia', 'Oceania', -37.8497, 144.9680, 1),
    ('Suzuka', 'Suzuka', 'Japan', 'Asia', 34.8431, 136.5419, 1),
    ('Shanghai', 'Shanghai', 'China', 'Asia', 31.3389, 121.2197, 1),
    ('Kuala Lumpur', 'Sepang', 'Malaysia', 'Asia', 2.7608, 101.7386, 1),
    ('Catalunya', 'Montmeló', 'Spain', 'Europe', 41.5700, 2.2611, 1),
    ('Spielberg', 'Spielberg', 'Austria', 'Europe', 47.2196, 14.7649, 1),
    ('Montreal', 'Montreal', 'Canada', 'North America', 45.5000, -73.5228, 1),
    ('Hockenheim', 'Hockenheim', 'Germany', 'Europe', 49.3278, 8.5658, 1),
    ('Marina Bay', 'Singapore', 'Singapore', 'Asia', 1.2914, 103.8645, 1),
    ('Austin', 'Austin', 'United States', 'North America', 30.1328, -97.6411, 1),
    ('Imola', 'Imola', 'Italy', 'Europe', 44.3439, 11.7167, 1),
    ('Mexico City', 'Mexico City', 'Mexico', 'North America', 19.4042, -99.0907, 1),
    ('Baku', 'Baku', 'Azerbaijan', 'Asia', 40.3725, 49.8533, 1),
    ('Istanbul', 'Istanbul', 'Turkey', 'Asia', 40.9517, 29.4050, 1),
    ('Magny Cours', 'Magny-Cours', 'France', 'Europe', 46.8642, 3.1633, 1),
    ('Spa-Francorchamps', 'Stavelot', 'Belgium', 'Europe', 50.4379, 5.9714, 1),
    ('Sochi', 'Sochi', 'Russia', 'Europe', 43.4057, 39.9578, 1),
    ('Indianapolis', 'Indianapolis', 'United States', 'North America', 39.7950, -86.2347, 1),
    ('Jeddah', 'Jeddah', 'Saudi Arabia', 'Asia', 21.6319, 39.1044, 1),
    ('Zandvoort', 'Zandvoort', 'Netherlands', 'Europe', 52.3888, 4.5409, 1),
    ('Valencia', 'Valencia', 'Spain', 'Europe', 39.4589, -0.3317, 1),
    ('Le Castellet', 'Le Castellet', 'France', 'Europe', 43.2500, 5.7917, 1),
    ('Lusail', 'Lusail', 'Qatar', 'Asia', 25.4914, 51.4542, 1),
    ('Yeongam County', 'Yeongam', 'South Korea', 'Asia', 34.7333, 126.4170, 1),
    ('Miami', 'Miami', 'United States', 'North America', 25.9581, -80.2389, 1),
    ('Uttar Pradesh', 'Greater Noida', 'India', 'Asia', 28.3500, 77.5350, 1),
    ('Monte Carlo', 'Monte Carlo', 'Monaco', 'Europe', 43.7347, 7.4206, 1),
    ('Las Vegas', 'Las Vegas', 'United States', 'North America', 36.1699, -115.1398, 1),
    ('Portimão', 'Portimão', 'Portugal', 'Europe', 37.2278, -8.6267, 1),
    ('Oyama', 'Oyama', 'Japan', 'Asia', 35.3711, 138.9278, 1),
    ('Nürburgring', 'Nürburg', 'Germany', 'Europe', 50.3356, 6.9475, 1),
    ('Yas Marina', 'Abu Dhabi', 'United Arab Emirates', 'Asia', 24.4672, 54.6031, 1),
    ('Mugello', 'Scarperia e San Piero', 'Italy', 'Europe',43.99, 11.3702, 1),
    ('Cape Town', 'Cape Town', 'South Africa', 'Africa', -33.9180, 18.4233, 0),
    ('Seoul', 'Seoul', 'South Korea', 'Asia', 37.5665, 126.9780, 0),
    ('Buenos Aires', 'Buenos Aires', 'Argentina', 'South America', -34.6037, -58.3816, 0),
    ('Chicago', 'Chicago', 'United States', 'North America', 41.8781, -87.6298, 0),
    ('Auckland', 'Auckland', 'New Zealand', 'Oceania', -36.8485, 174.7633, 0),
    ('Riyadh', 'Riyadh', 'Saudi Arabia', 'Asia', 24.7136, 46.6753, 0),
    ('Lagos', 'Lagos', 'Nigeria', 'Africa', 6.5244, 3.3792, 0),
    ('Oslo', 'Oslo', 'Norway', 'Europe', 59.9139, 10.7522, 0),
    ('Kuala Terengganu', 'Kuala Terengganu', 'Malaysia', 'Asia', 5.3290, 103.1408, 0),
    ('Vancouver', 'Vancouver', 'Canada', 'North America', 49.2827, -123.1207, 0)
    ]

'''
Cape Town: Often suggested for African representation in F1; great scenery and infrastructure.
Seoul: Huge tech hub with existing motorsport interest, could tie well with the Korean automotive industry.
Buenos Aires: Historic F1 location, potential for a return.
Chicago: Fills the gap in central U.S. with a global city profile.
Auckland: Expands Oceania representation beyond Australia.
Riyadh: Already hosts Formula E; could evolve into full F1 venue.
Lagos: Major economic center in Africa; bold long-term candidate.
Oslo: Scandinavia has no F1 GP currently — great fan base.
Kuala Terengganu: A hypothetical Malaysian alternative to Sepang.
Vancouver: Canada\'s west coast city with scenic urban race potential.
'''

# Insert unique circuit data into the fone_geography table
cursor.executemany('''
INSERT INTO fone_geography (circuit, city, country, continent, latitude, longitude, existing)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', geography_data)

# Add a geo_id column to the fone_calendar table and establish the foreign key relationship
cursor.execute('ALTER TABLE fone_calendar ADD COLUMN geo_id INTEGER')

# Update geo_id using the same_circuits dictionary
for key, values in same_circuits.items():
    cursor.execute('''
    UPDATE fone_calendar
    SET geo_id = (
        SELECT id FROM fone_geography
        WHERE fone_geography.circuit = ?
    )
    WHERE fone_calendar.circuit IN ({placeholders})
    '''.format(placeholders=','.join('?' * len(values))), [key] + values)

# Update geo_id for circuits that directly match
cursor.execute('''
UPDATE fone_calendar
SET geo_id = (
    SELECT id FROM fone_geography
    WHERE fone_geography.circuit = fone_calendar.circuit
)
WHERE geo_id IS NULL
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("fone_geography table created and linked to fone_calendar successfully.")

fone_geography table created and linked to fone_calendar successfully.


In [5]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the travel_logistic table
cursor.execute('''
CREATE TABLE IF NOT EXISTS travel_logistic (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    from_id INTEGER NOT NULL,
    from_circuit TEXT NOT NULL,
    to_id INTEGER NOT NULL,
    to_circuit TEXT NOT NULL,
    distance_km REAL,
    transport_mode TEXT
)
''')

# Fetch all circuits from the fone_geography table
cursor.execute('SELECT id, circuit FROM fone_geography')
circuits = cursor.fetchall()

# Generate all possible combinations of circuits
travel_data = []
for from_circuit in circuits:
    for to_circuit in circuits:
        if from_circuit[0] != to_circuit[0]:  # Avoid same circuit combinations
            travel_data.append((from_circuit[0], from_circuit[1], to_circuit[0], to_circuit[1]))

# Insert the combinations into the travel_logistic table
cursor.executemany('''
INSERT INTO travel_logistic (from_id, from_circuit, to_id, to_circuit)
VALUES (?, ?, ?, ?)
''', travel_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print(f"{len(travel_data)} records inserted into the travel_logistic table.")

2256 records inserted into the travel_logistic table.


In [6]:
import math

# Function to calculate distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Fetch latitude and longitude for all circuits
cursor.execute('SELECT id, latitude, longitude FROM fone_geography')
geo_data = {row[0]: (row[1], row[2]) for row in cursor.fetchall()}

# Update the distance_km column in the travel_logistic table
for from_id, from_circuit, to_id, to_circuit in travel_data:
    if from_id in geo_data and to_id in geo_data:
        lat1, lon1 = geo_data[from_id]
        lat2, lon2 = geo_data[to_id]
        distance = haversine(lat1, lon1, lat2, lon2)
        cursor.execute('''
        UPDATE travel_logistic
        SET distance_km = ?
        WHERE from_id = ? AND to_id = ?
        ''', (distance, from_id, to_id))

# Commit changes and close the connection
conn.commit()
conn.close()

print("Distances calculated and updated successfully.")

Distances calculated and updated successfully.


In [12]:
# Define transportation modes
# Heuristics for transport modes that requires air transport based on geography
def needs_air_transport(from_country, to_country, from_continent, to_continent):
    island_countries = [
        'United Kingdom', 'Japan', 'Australia', 'Singapore', 'New Zealand',
        'Malaysia', 'Indonesia'
    ]
    isolated_countries = [
        'South Africa', 'Nigeria', 'South Korea'
    ]
    if from_continent != to_continent:
        return True
    if from_country in island_countries or to_country in island_countries:
        return True
    if from_country in isolated_countries or to_country in isolated_countries:
        return True
    if 'Oceania' in (from_continent, to_continent):
        return True
    return False


In [ ]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Add the new column 'needs_air' to the travel_logistic table
cursor.execute('ALTER TABLE travel_logistic ADD COLUMN needs_air BOOLEAN')

# Fetch necessary data from the travel_logistic and fone_geography tables
cursor.execute('''
SELECT tl.id, fg1.country AS from_country, fg2.country AS to_country,
    fg1.continent AS from_continent, fg2.continent AS to_continent
FROM travel_logistic tl
JOIN fone_geography fg1 ON tl.from_id = fg1.id
JOIN fone_geography fg2 ON tl.to_id = fg2.id
''')
travel_records = cursor.fetchall()

# Update the travel_logistic table with the air transport requirement
for record in travel_records:
    travel_id, from_country, to_country, from_continent, to_continent = record
    air_transport = needs_air_transport(from_country, to_country, from_continent, to_continent)
    cursor.execute('''
    UPDATE travel_logistic
    SET needs_air = ?
    WHERE id = ?
    ''', (air_transport, travel_id))

# Commit changes and close the connection
conn.commit()
conn.close()

print("needs_air column added and updated successfully in the travel_logistic table.")


needs_air column added and updated successfully in the travel_logistic table.


In [9]:
#using Openroute Service API to calculate distances for trucks
import openrouteservice
from openrouteservice.exceptions import ApiError
import time

def get_truck_route_info(origin, destination):
    """
    Given two (lat, lon) tuples, return distance in km and truck accessibility.
    
    Args:
        origin (tuple): (lat, lon) for the start point.
        destination (tuple): (lat, lon) for the end point.
        
    Returns:
        dict: {
            'distance_km': float or None,
            'truck_accessible': bool
        }
    """
    client = openrouteservice.Client(key='5b3ce3597851110001cf624867038ce71ba14d2c9cbdca5216627a22')
    
    try:
        route = client.directions(
            coordinates=[(origin[1], origin[0]), (destination[1], destination[0])],
            profile='driving-hgv',
            format='geojson'
        )
        
        distance_m = route['features'][0]['properties']['segments'][0]['distance']
        return {
            'distance_km': round(distance_m / 1000, 2),
            'truck_accessible': True
        }
    
    except ApiError as e:
        # If the API can't find a route for trucks, it will raise an error
        print(f"Route error: {e}")
        return {
            'distance_km': None,
            'truck_accessible': False
        }

def rate_limited_get_truck_route_info(origin, destination, requests_per_minute=39):
    """
    Wrapper function to enforce rate limiting for API requests.
    
    Args:
        origin (tuple): (lat, lon) for the start point.
        destination (tuple): (lat, lon) for the end point.
        requests_per_minute (int): Maximum number of requests allowed per minute.
        
    Returns:
        dict: Result from get_truck_route_info.
    """
    delay = 60 / requests_per_minute  # Calculate delay between requests
    time.sleep(delay)  # Enforce delay
    return get_truck_route_info(origin, destination)

In [10]:
#testing api

magny_course = (46.8642, 3.1633)
monza = (45.6156, 9.2811)

result = get_truck_route_info(magny_course, monza)
print(result)

{'distance_km': 713.43, 'truck_accessible': True}


In [11]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Add the new columns to the travel_logistic table
cursor.execute('ALTER TABLE travel_logistic ADD COLUMN truck_distance_km REAL')
cursor.execute('ALTER TABLE travel_logistic ADD COLUMN truck_viable BOOLEAN')

# Fetch necessary data from the travel_logistic and fone_geography tables
cursor.execute('''
SELECT tl.id, fg1.latitude AS from_lat, fg1.longitude AS from_lon,
    fg2.latitude AS to_lat, fg2.longitude AS to_lon, tl.needs_air
FROM travel_logistic tl
JOIN fone_geography fg1 ON tl.from_id = fg1.id
JOIN fone_geography fg2 ON tl.to_id = fg2.id
''')
travel_records = cursor.fetchall()

total_records = len(travel_records)
print(f"Total records to process: {total_records}")

# Update the travel_logistic table with truck route information
for index, record in enumerate(travel_records, start=1):
    travel_id, from_lat, from_lon, to_lat, to_lon, needs_air = record
    if not needs_air:  # Only process records where needs_air = 0
        origin = (from_lat, from_lon)
        destination = (to_lat, to_lon)
        route_info = rate_limited_get_truck_route_info(origin, destination)
        cursor.execute('''
        UPDATE travel_logistic
        SET truck_distance_km = ?, truck_viable = ?
        WHERE id = ?
        ''', (route_info['distance_km'], route_info['truck_accessible'], travel_id))
    
    # Print progress
    print(f"Processed {index}/{total_records} records")

# Commit changes and close the connection
conn.commit()
conn.close()

print("truck_distance_km and truck_viable columns updated successfully in the travel_logistic table.")

Total records to process: 2256
Processed 1/2256 records
Processed 2/2256 records
Processed 3/2256 records
Processed 4/2256 records
Processed 5/2256 records
Processed 6/2256 records
Processed 7/2256 records
Processed 8/2256 records
Processed 9/2256 records
Processed 10/2256 records
Processed 11/2256 records
Processed 12/2256 records
Processed 13/2256 records
Processed 14/2256 records
Processed 15/2256 records
Processed 16/2256 records
Processed 17/2256 records
Processed 18/2256 records
Processed 19/2256 records
Processed 20/2256 records
Processed 21/2256 records
Processed 22/2256 records
Processed 23/2256 records
Processed 24/2256 records
Processed 25/2256 records
Processed 26/2256 records
Processed 27/2256 records
Processed 28/2256 records
Processed 29/2256 records
Processed 30/2256 records
Processed 31/2256 records
Processed 32/2256 records
Processed 33/2256 records
Processed 34/2256 records
Processed 35/2256 records
Processed 36/2256 records
Processed 37/2256 records
Processed 38/225

c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Processed 118/2256 records
Processed 119/2256 records
Processed 120/2256 records
Processed 121/2256 records
Processed 122/2256 records
Processed 123/2256 records
Processed 124/2256 records
Processed 125/2256 records
Processed 126/2256 records
Processed 127/2256 records
Processed 128/2256 records
Processed 129/2256 records
Processed 130/2256 records
Route error: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 11.3725000 43.9975000.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1745229409862}})
Processed 131/2256 records
Processed 132/2256 records
Processed 133/2256 records
Processed 134/2256 records
Processed 135/2256 records
Processed 136/2256 records
Processed 137/2256 records
Processed 138/2256 records
Processed 139/2256 records
Processed 140/2256 records
Processed 141/2256 records
Processed 142/2256 records
Processed 143/2256 recor

c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Route error: 400 ({'error': {'code': 2004, 'message': 'Request parameters exceed the server configuration limits. The approximated route distance must not be greater than 6000000.0 meters.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1745229458103}})
Processed 365/2256 records
Processed 366/2256 records
Processed 367/2256 records
Processed 368/2256 records
Processed 369/2256 records
Processed 370/2256 records
Processed 371/2256 records
Route error: 400 ({'error': {'code': 2004, 'message': 'Request parameters exceed the server configuration limits. The approximated route distance must not be greater than 6000000.0 meters.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1745229464392}})
Processed 372/2256 records
Processed 373/2256 records
Processed 374/2256 records
Processed 375/2256 records
Processed 376/2256 records
Processed 377/2256 records
Pro

c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\massi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Processed 1022/2256 records
Processed 1023/2256 records
Route error: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 11.3725000 43.9975000.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1745229852563}})
Processed 1024/2256 records
Processed 1025/2256 records
Processed 1026/2256 records
Processed 1027/2256 records
Processed 1028/2256 records
Processed 1029/2256 records
Processed 1030/2256 records
Processed 1031/2256 records
Processed 1032/2256 records
Processed 1033/2256 records
Processed 1034/2256 records
Processed 1035/2256 records
Processed 1036/2256 records
Processed 1037/2256 records
Processed 1038/2256 records
Processed 1039/2256 records
Processed 1040/2256 records
Processed 1041/2256 records
Processed 1042/2256 records
Processed 1043/2256 records
Processed 1044/2256 records
Processed 1045/2256 records
Route error: 404 ({'error':

In [ ]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# FIXING SOME LATITUDE AND LONGITUDE VALUES
# The following circuits have incorrect latitude and longitude values
# Update the latitude and longitude for the specified circuits
updates = [
    ('Mugello', 43.9900, 11.3702),
    ('Indianapolis', 39.7876, -86.2392),
    ('Spa-Francorchamps', 50.4455, 5.9708),
    ('Shanghai', 31.3303, 121.2246),
    ('Hockenheim', 49.3249, 8.5694)
]

for circuit, latitude, longitude in updates:
    cursor.execute('''
    UPDATE fone_geography
    SET latitude = ?, longitude = ?
    WHERE circuit = ?
    ''', (latitude, longitude, circuit))

# Commit changes and close the connection
conn.commit()
conn.close()

print("Records updated successfully in the fone_geography table.")

Records updated successfully in the fone_geography table.


In [22]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Fetch necessary data for records where needs_air = 0 and truck_viable = 0
cursor.execute('''
SELECT tl.id, fg1.latitude AS from_lat, fg1.longitude AS from_lon,
    fg2.latitude AS to_lat, fg2.longitude AS to_lon
FROM travel_logistic tl
JOIN fone_geography fg1 ON tl.from_id = fg1.id
JOIN fone_geography fg2 ON tl.to_id = fg2.id
WHERE tl.needs_air = 0 AND (tl.truck_viable IS NULL OR tl.truck_viable = 0)
''')
subset_records = cursor.fetchall()

total_subset_records = len(subset_records)
print(f"Total subset records to process: {total_subset_records}")

# Update the travel_logistic table with truck route information for the subset
for index, record in enumerate(subset_records, start=1):
    travel_id, from_lat, from_lon, to_lat, to_lon = record
    origin = (from_lat, from_lon)
    destination = (to_lat, to_lon)
    route_info = rate_limited_get_truck_route_info(origin, destination)
    cursor.execute('''
    UPDATE travel_logistic
    SET truck_distance_km = ?, truck_viable = ?
    WHERE id = ?
    ''', (route_info['distance_km'], route_info['truck_accessible'], travel_id))
    
    # Print progress
    print(f"Processed {index}/{total_subset_records} records")

# Commit changes and close the connection
conn.commit()
conn.close()

print("Subset of travel_logistic table updated successfully.")

Total subset records to process: 30
Processed 1/30 records
Processed 2/30 records
Route error: 400 ({'error': {'code': 2004, 'message': 'Request parameters exceed the server configuration limits. The approximated route distance must not be greater than 6000000.0 meters.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1745250212289}})
Processed 3/30 records
Route error: 400 ({'error': {'code': 2004, 'message': 'Request parameters exceed the server configuration limits. The approximated route distance must not be greater than 6000000.0 meters.'}, 'info': {'engine': {'build_date': '2025-04-10T21:25:30Z', 'graph_version': '1', 'version': '9.1.2'}, 'timestamp': 1745250214311}})
Processed 4/30 records
Route error: 400 ({'error': {'code': 2004, 'message': 'Request parameters exceed the server configuration limits. The approximated route distance must not be greater than 6000000.0 meters.'}, 'info': {'engine': {'build_date': 

In [23]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

#assuming that needs_air is 1 for all remaining records where truck_viable is 0
# Update records in the travel_logistic table
cursor.execute('''
UPDATE travel_logistic
SET needs_air = 1
WHERE needs_air = 0 AND truck_viable = 0
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("Records updated successfully where needs_air = 0 and truck_viable = 0.")

Records updated successfully where needs_air = 0 and truck_viable = 0.


In [24]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Define the isolated countries
isolated_countries = ['South Africa', 'Nigeria', 'South Korea']

# Update the travel_logistic table for routes involving isolated countries
cursor.execute('''
UPDATE travel_logistic
SET truck_viable = 0, truck_distance_km = NULL
WHERE from_id IN (
    SELECT id FROM fone_geography WHERE country IN ({placeholders})
)
OR to_id IN (
    SELECT id FROM fone_geography WHERE country IN ({placeholders})
)
'''.format(placeholders=','.join('?' * len(isolated_countries))), isolated_countries * 2)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Updated truck_viable and truck_distance_km for isolated countries in the travel_logistic table.")

Updated truck_viable and truck_distance_km for isolated countries in the travel_logistic table.


[2025 Estimates]
🚛 Truck Emissions (per tonne-km)
From sources:

Diesel trucks: ~100 g/tonne-km

HVO100 biofuel trucks: ~17 g/tonne-km (83% reduction from diesel)

✈️ Air Freight Emissions (per tonne-km)
From sources:

Conventional Jet Fuel: ~500 g/tonne-km

Sustainable Aviation Fuel (SAF): ~100 g/tonne-km (80% reduction)
Mode	Sustainable Fuel Share	Conventional Fuel Share
Truck	60–70% HVO100	        30–40% Diesel
Air	    ~15–20% SAF (2023)	    ~80–85% Jet Fuel
Let’s use conservative estimates:

Truck: 70% HVO100, 30% Diesel

Air: 20% SAF, 80% Jet Fuel

Truck	
~42 g/tonne-km	
70% HVO100, 30% diesel

Air	
~420 g/tonne-km	
20% SAF, 80% jet fuel

F1 typically moves ~1,000 tonnes of equipment.

Emission Factor
Air: ~420 g CO₂ / tonne-km
Truck: ~42 g CO₂ / tonne-km

Total CO₂ (kg) = Distance (km) × Cargo Mass (tonnes) × Emission Factor (g/tonne-km) ÷ 1000

truck_emissions = truck_distance_km x 1000 x 420 / 1000
air_emissions = distance_km x 1000 x 42 / 1000




In [31]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Add new columns for emissions if they don't exist
cursor.execute('ALTER TABLE travel_logistic ADD COLUMN truck_emissions REAL')
cursor.execute('ALTER TABLE travel_logistic ADD COLUMN air_emissions REAL')

# Update the emissions based on the distance values
cursor.execute('''
UPDATE travel_logistic
SET truck_emissions = truck_distance_km * 1000 * 42 / 1000,
    air_emissions = distance_km * 1000 * 420 / 1000

''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("Emissions calculated and updated successfully.")

OperationalError: duplicate column name: truck_emissions

In [33]:
triple_header_rules: dict[str, object] = {
    "max_per_season": 2,
    "min_distance_km": 0,
    "max_distance_km": 3000,
    "consecutive_weekends": True,
    "avoid_if": ["extreme_heat", "visa_restrictions", "cross_continent"]
}

In [ ]:
# import pandas as pd
# import sqlite3

# # Load the fone_geography.csv file
# fone_geography_path = 'data/fone_geography.csv'
# fone_geography_df = pd.read_csv(fone_geography_path)

# # Load the Costraints.xlsx file
# constraints_path = 'data/Costraints.xlsx'
# try:
#     constraints_df = pd.read_excel(constraints_path)
# except FileNotFoundError:
#     print(f"Error: The file '{constraints_path}' was not found. Please ensure it exists in the specified directory.")
#     constraints_df = pd.DataFrame()  # Create an empty DataFrame as a fallback

# # Merge the data based on the 'id' column and drop duplicated columns from constraints_df
# updated_df = pd.merge(fone_geography_df, constraints_df, on='id', how='left')

# # Reopen the database connection
# conn = sqlite3.connect('planet_fone.db')

# # Replace the fone_geography table with the updated data
# updated_df.to_sql('fone_geography', conn, if_exists='replace', index=False)

# # Commit changes and close the connection
# conn.commit()
# conn.close()

# print("fone_geography table updated successfully in the database.")


fone_geography table updated successfully in the database.


In [ ]:
# import sqlite3

# # Reopen the database connection
# conn = sqlite3.connect('planet_fone.db')
# cursor = conn.cursor()

# # Step 1: Create a new table excluding the unwanted columns
# cursor.execute('''
# CREATE TABLE IF NOT EXISTS fone_geography_new AS
# SELECT id, circuit_x, city_x, country_x, continent, latitude, longitude, existing, months_to_avoid, traditional_months, notes
# FROM fone_geography
# ''')

# # Step 2: Drop the old table
# cursor.execute('DROP TABLE fone_geography')

# # Step 3: Rename the new table to the original table name
# cursor.execute('ALTER TABLE fone_geography_new RENAME TO fone_geography')

# # Commit changes and close the connection
# conn.commit()
# conn.close()

# print("Columns removed successfully from fone_geography table.")

Columns removed successfully from fone_geography table.


based on data in fone_calendar in the db, add 2 columns 1 for inbound_route and one for the outbound_route
the rules are:
a race is a record in the fone_calendar table
previous race is the previous one by column date
next race is the next one by column date
for inbound_route
    first race of every year is null
    contains the id of the table travel_logistic where from_id is equal to the previous race's geo_id and the to_id is equal to the same race geo_id

for outbound_route
    last race of every year is null
    contains the id of the table travel_logistic where from_id is equal to this race geo_id and the to_id is equal to the next race geo_id


In [5]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Add the new columns to the fone_calendar table
cursor.execute('ALTER TABLE fone_calendar ADD COLUMN inbound_route INTEGER')
cursor.execute('ALTER TABLE fone_calendar ADD COLUMN outbound_route INTEGER')

# Fetch all races ordered by year and date
cursor.execute('''
SELECT id, year, date, geo_id
FROM fone_calendar
ORDER BY year, date
''')
races = cursor.fetchall()

# Iterate through the races and calculate inbound and outbound routes
for i, race in enumerate(races):
    race_id, year, date, geo_id = race

    # Determine the previous and next races
    previous_race = races[i - 1] if i > 0 and races[i - 1][1] == year else None
    next_race = races[i + 1] if i < len(races) - 1 and races[i + 1][1] == year else None

    # Calculate inbound_route
    if previous_race:
        prev_geo_id = previous_race[3]
        cursor.execute('''
        SELECT id FROM travel_logistic
        WHERE from_id = ? AND to_id = ?
        ''', (prev_geo_id, geo_id))
        inbound_route = cursor.fetchone()
        inbound_route_id = inbound_route[0] if inbound_route else None
    else:
        inbound_route_id = None

    # Calculate outbound_route
    if next_race:
        next_geo_id = next_race[3]
        cursor.execute('''
        SELECT id FROM travel_logistic
        WHERE from_id = ? AND to_id = ?
        ''', (geo_id, next_geo_id))
        outbound_route = cursor.fetchone()
        outbound_route_id = outbound_route[0] if outbound_route else None
    else:
        outbound_route_id = None

    # Update the fone_calendar table with the calculated routes
    cursor.execute('''
    UPDATE fone_calendar
    SET inbound_route = ?, outbound_route = ?
    WHERE id = ?
    ''', (inbound_route_id, outbound_route_id, race_id))

# Commit changes and close the connection
conn.commit()
conn.close()

print("Inbound and outbound routes updated successfully in the fone_calendar table.")

Inbound and outbound routes updated successfully in the fone_calendar table.


In [6]:
import os
import pandas as pd

# Create the 'data' folder if it doesn't exist
os.makedirs('data', exist_ok=True)

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')

# Fetch all table names from the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Export each table to a CSV file
for table_name in tables:
    table_name = table_name[0]
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    csv_file_path = os.path.join('data', f"{table_name}.csv")
    df.to_csv(csv_file_path, index=False)
    print(f"Exported {table_name} to {csv_file_path}")

# Close the connection
conn.close()

print("All tables exported successfully.")

Exported calendar to data\calendar.csv
Exported sqlite_sequence to data\sqlite_sequence.csv
Exported fone_calendar to data\fone_calendar.csv
Exported travel_logistic to data\travel_logistic.csv
Exported fone_geography to data\fone_geography.csv
All tables exported successfully.
